In [1]:
import os
import json
import pandas as pd
import numpy as np

### Making a dataframe from the results folders to handle the data

In [ ]:
# params
path = '/media/c1046372/Expansion/Thesis GY/3. Analyzed files'
df_in = pd.read_excel('Positions.xlsx')
df_out = df_in[df_in['Quality'] == 'Very good'].copy().reset_index(drop=True)

df_out.loc[:,["area_max"]] = np.nan
# Initialize the columns as type 'object' to store arrays/lists
df_out['radius'] = pd.Series(dtype='object') # colony radius in time
df_out['area'] = pd.Series(dtype='object') # colony area in time

# growth fitting
df_out.loc[:,["gomp_params"]] = np.nan # parameters fitting Gompertz
df_out.loc[:,["logistic_params"]] = np.nan # parameters fitting Logistic
df_out.loc[:,["t_m_gomp"]] = np.nan # time at which mu is maximum in Gompertz
df_out.loc[:,["t_m_logistic"]] = np.nan # time at which mu is maximum in Logistic

# velocity field fitting
df_out.loc[:,["r0"]] = np.nan
df_out.loc[:,["vel_start"]] = np.nan #frame at which the velocimetry starts
df_out.loc[:,["vel_stop"]] = np.nan #frame at which the velocimetry ends
df_out.loc[:,["vel_fit_start"]] = np.nan #frame at which the fitting starts
df_out.loc[:,["vel_fit_stop"]] = np.nan #frame at which the fitting ends
df_out['mu0'] = pd.Series(dtype='object')

# wave speed fitting
df_out.loc[:,["ws_params"]] = np.nan

# Display the head of the DataFrame to check the assignment
df_out.head()

#### Extract area and radius endpoints, as well as all dynamics

In [ ]:
# extract area and radius from each position's results and store it in a dataframe
scopes = os.listdir(path)
for s in scopes:
    scope_dates = os.listdir(os.path.join(path,s))
    scope_dates.sort()
    for d in scope_dates:
        date = datetime.strptime(d, '%Y_%m_%d')
        data = df_out[(df_out.Date == date) & (df_out.Machine == s)]
        pos = data.Position.unique()
        path_results = os.path.join(path,s,d,'results')
        for p in pos:
            path_pos = os.path.join(path_results,f'pos{p}')
            area_path = os.path.join(path_pos, 'area.npy')
            radius_path = os.path.join(path_pos, 'radius.npy')

            ridx = df_out[(df_out.Date == date) & (df_out.Machine==s) & (df_out.Position == p)].index[0]

            # load area and radius
            radius = np.load(radius_path)  
            area = np.load(area_path)
            outn = 'data_processed.xlsx'

            # quantities are not filtered
            df_out.at[ridx, 'radius'] = radius
            df_out.at[ridx, 'area'] = area

In [ ]:
df_out.to_excel(outn, index=False)